# Module 4: Machine Learning for Time Series Forecasting

## Learning Objectives

After completing this module, you will:
1. Understand when to use ML vs. statistical methods
2. Engineer time series features (lags, rolling statistics, seasonal)
3. Properly split time series data for validation
4. Build Random Forest, XGBoost, and LightGBM forecasters
5. Perform hyperparameter tuning
6. Interpret feature importance
7. Evaluate and compare ML models

## ML vs. Statistical Methods

| Aspect | Statistical (ARIMA) | Machine Learning |
|--------|---------------------|------------------|
| Interpretability | High | Medium |
| Speed | Fast | Fast-Medium |
| Feature Engineering | Limited | Extensive |
| Multiple External Variables | Difficult | Easy |
| Parameter Tuning | Moderate | Extensive |
| Automation | Good | Good |
| Uncertainty Quantification | Built-in | Via ensembles |
| Large Datasets | Works | Thrives |

# Module 4: Machine Learning for Forecasting
## Mini-Project 4: Complete ML Forecasting Pipeline

**Objective:** Build and compare multiple ML models for time series forecasting with comprehensive feature engineering.

**Dataset:** Airline Passengers (monthly data, 1949-1960)

**Learning Outcomes:**
- Engineer features from time series data
- Implement 5+ ML models
- Perform time-aware model selection
- Compare ML vs statistical methods
- Analyze feature importance and model diagnostics

## Section 4.1: Setup and Feature Engineering

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load data
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
df = pd.read_csv(url)
df['Time'] = pd.date_range(start='1949-01', periods=len(df), freq='MS')
df = df.set_index('Time')

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}\n")

# Prepare train/test split (80/20)
n = len(df)
split_idx = int(0.8 * n)
train_df = df.iloc[:split_idx].copy()
test_df = df.iloc[split_idx:].copy()

print(f"Train set: {len(train_df)} observations")
print(f"Test set: {len(test_df)} observations")

In [ ]:
# Feature Engineering for Time Series
print("\n" + "=" * 70)
print("FEATURE ENGINEERING")
print("=" * 70)

def create_features(data, lags=[1, 3, 6, 12], rolling_windows=[3, 6, 12]):
    """Create lag and rolling statistics features"""
    df = data.copy()
    
    # Lag features
    for lag in lags:
        df[f'lag_{lag}'] = df['Passengers'].shift(lag)
    
    # Rolling mean features
    for window in rolling_windows:
        df[f'rolling_mean_{window}'] = df['Passengers'].rolling(window).mean()
        df[f'rolling_std_{window}'] = df['Passengers'].rolling(window).std()
    
    # Seasonal feature (month of year)
    df['month'] = df.index.month
    df['quarter'] = df.index.quarter
    
    # Time index (trend)
    df['time_index'] = np.arange(len(df))
    
    return df

# Create features for full dataset
df_features = create_features(df)
print(f"\nOriginal columns: {df.columns.tolist()}")
print(f"After feature engineering: {df_features.shape[1]} columns")
print(f"\nFirst few rows with features:")
print(df_features.head(15))

# Remove rows with NaN (from lags/rolling windows)
df_features_clean = df_features.dropna()
print(f"\nAfter removing NaNs: {df_features_clean.shape[0]} observations")

# Resplit train/test
split_idx_clean = int(0.8 * len(df_features_clean))
X_train = df_features_clean.iloc[:split_idx_clean].drop('Passengers', axis=1)
y_train = df_features_clean.iloc[:split_idx_clean]['Passengers']
X_test = df_features_clean.iloc[split_idx_clean:].drop('Passengers', axis=1)
y_test = df_features_clean.iloc[split_idx_clean:]['Passengers']

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nFeatures used:\n{X_train.columns.tolist()}")

## Section 4.2: Random Forest for Time Series

In [ ]:
# Random Forest
print("\n" + "=" * 70)
print("RANDOM FOREST REGRESSOR")
print("=" * 70)

# Train base model
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)
rf_pred_test = rf_model.predict(X_test)

# Metrics
rf_mae = mean_absolute_error(y_test, rf_pred_test)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred_test))
rf_mape = mean_absolute_percentage_error(y_test, rf_pred_test)

print(f"\nRandom Forest Performance:")
print(f"  MAE:  {rf_mae:.2f}")
print(f"  RMSE: {rf_rmse:.2f}")
print(f"  MAPE: {rf_mape:.2%}")

# Feature importance
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nTop 10 Most Important Features:")
print(importance_df.head(10).to_string(index=False))

# Visualize importance
fig, ax = plt.subplots(figsize=(12, 6))
top_features = importance_df.head(12)
ax.barh(range(len(top_features)), top_features['Importance'])
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['Feature'])
ax.set_xlabel('Feature Importance')
ax.set_title('Random Forest - Top 12 Feature Importance', fontweight='bold', fontsize=12)
ax.invert_yaxis()
plt.tight_layout()
plt.show()

# Hyperparameter tuning
print(f"\nHyperparameter Tuning for Random Forest...")
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [5, 10]
}

rf_grid = GridSearchCV(
    RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

rf_grid.fit(X_train, y_train)
rf_best_model = rf_grid.best_estimator_

print(f"\nBest parameters: {rf_grid.best_params_}")
rf_best_pred = rf_best_model.predict(X_test)
rf_best_rmse = np.sqrt(mean_squared_error(y_test, rf_best_pred))
print(f"Best model RMSE: {rf_best_rmse:.2f}")
print(f"Improvement: {(1 - rf_best_rmse/rf_rmse)*100:.1f}%")

## Section 4.3: XGBoost and LightGBM

In [ ]:
# XGBoost
print("\n" + "=" * 70)
print("XGBOOST REGRESSOR")
print("=" * 70)

xgb_model = XGBRegressor(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=0
)

xgb_model.fit(X_train, y_train, early_stopping_rounds=10,
              eval_set=[(X_test, y_test)], verbose=False)

xgb_pred = xgb_model.predict(X_test)
xgb_mae = mean_absolute_error(y_test, xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
xgb_mape = mean_absolute_percentage_error(y_test, xgb_pred)

print(f"\nXGBoost Performance:")
print(f"  MAE:  {xgb_mae:.2f}")
print(f"  RMSE: {xgb_rmse:.2f}")
print(f"  MAPE: {xgb_mape:.2%}")

# LightGBM
print(f"\nLightGBM Regressor")
print("-" * 70)

lgb_model = LGBMRegressor(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)

lgb_model.fit(X_train, y_train)
lgb_pred = lgb_model.predict(X_test)
lgb_mae = mean_absolute_error(y_test, lgb_pred)
lgb_rmse = np.sqrt(mean_squared_error(y_test, lgb_pred))
lgb_mape = mean_absolute_percentage_error(y_test, lgb_pred)

print(f"\nLightGBM Performance:")
print(f"  MAE:  {lgb_mae:.2f}")
print(f"  RMSE: {lgb_rmse:.2f}")
print(f"  MAPE: {lgb_mape:.2%}")

# Comparison
print(f"\n" + "=" * 70)
print("ML MODELS COMPARISON")
print("=" * 70)

comparison_data = {
    'Model': ['Random Forest', 'RF (Tuned)', 'XGBoost', 'LightGBM'],
    'MAE': [rf_mae, mean_absolute_error(y_test, rf_best_pred), xgb_mae, lgb_mae],
    'RMSE': [rf_rmse, rf_best_rmse, xgb_rmse, lgb_rmse],
    'MAPE': [rf_mape, mean_absolute_percentage_error(y_test, rf_best_pred), xgb_mape, lgb_mape]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('RMSE')
print(f"\n{comparison_df.to_string(index=False)}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE comparison
ax = axes[0]
rmse_values = [rf_rmse, rf_best_rmse, xgb_rmse, lgb_rmse]
model_names = ['RF Base', 'RF Tuned', 'XGBoost', 'LightGBM']
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(rmse_values)))
ax.bar(model_names, rmse_values, color=colors)
ax.set_ylabel('RMSE (Lower is Better)')
ax.set_title('ML Models RMSE Comparison', fontweight='bold', fontsize=12)
ax.grid(alpha=0.3, axis='y')
for i, v in enumerate(rmse_values):
    ax.text(i, v + 0.5, f'{v:.2f}', ha='center', fontweight='bold')

# Actual vs predicted for best model
ax = axes[1]
best_idx = comparison_df['RMSE'].idxmin()
best_model_name = comparison_df.iloc[best_idx]['Model']

if 'LightGBM' in best_model_name:
    best_pred = lgb_pred
elif 'XGBoost' in best_model_name:
    best_pred = xgb_pred
elif 'RF Tuned' in best_model_name:
    best_pred = rf_best_pred
else:
    best_pred = rf_pred_test

ax.plot(y_test.values, 'o-', label='Actual', linewidth=2, markersize=6)
ax.plot(best_pred, 's--', label=f'{best_model_name} Forecast', linewidth=2, markersize=5)
ax.set_xlabel('Test Set Index')
ax.set_ylabel('Passengers')
ax.set_title(f'Best Model: {best_model_name}', fontweight='bold', fontsize=12)
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("\n" + "=" * 60)
print("MODEL 2: Random Forest")
print("=" * 60)

# Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)  # Note: No scaling needed for tree models
y_pred_rf = rf_model.predict(X_test)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf)

models['Random Forest'] = rf_model
predictions['Random Forest'] = y_pred_rf
metrics['Random Forest'] = {
    'MAE': mae_rf,
    'RMSE': rmse_rf,
    'MAPE': mape_rf
}

print(f"MAE:  {mae_rf:.4f}")
print(f"RMSE: {rmse_rf:.4f}")
print(f"MAPE: {mape_rf:.4f}")

In [ ]:
print("\n" + "=" * 60)
print("MODEL 3: XGBoost")
print("=" * 60)

# XGBoost
xgb_model = xgb.XGBRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)
y_pred_xgb = xgb_model.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mape_xgb = mean_absolute_percentage_error(y_test, y_pred_xgb)

models['XGBoost'] = xgb_model
predictions['XGBoost'] = y_pred_xgb
metrics['XGBoost'] = {
    'MAE': mae_xgb,
    'RMSE': rmse_xgb,
    'MAPE': mape_xgb
}

print(f"MAE:  {mae_xgb:.4f}")
print(f"RMSE: {rmse_xgb:.4f}")
print(f"MAPE: {mape_xgb:.4f}")

In [ ]:
print("\n" + "=" * 60)
print("MODEL 4: LightGBM")
print("=" * 60)

# LightGBM
lgb_model = lgb.LGBMRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    num_leaves=31,
    random_state=42,
    verbose=-1
)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)

mae_lgb = mean_absolute_error(y_test, y_pred_lgb)
rmse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
mape_lgb = mean_absolute_percentage_error(y_test, y_pred_lgb)

models['LightGBM'] = lgb_model
predictions['LightGBM'] = y_pred_lgb
metrics['LightGBM'] = {
    'MAE': mae_lgb,
    'RMSE': rmse_lgb,
    'MAPE': mape_lgb
}

print(f"MAE:  {mae_lgb:.4f}")
print(f"RMSE: {rmse_lgb:.4f}")
print(f"MAPE: {mape_lgb:.4f}")

In [ ]:
print("\n" + "=" * 60)
print("MODEL 5: Support Vector Regression")
print("=" * 60)

# Support Vector Regression
svr_model = SVR(kernel='rbf', C=100, epsilon=0.1)
svr_model.fit(X_train_scaled, y_train)
y_pred_svr = svr_model.predict(X_test_scaled)

mae_svr = mean_absolute_error(y_test, y_pred_svr)
rmse_svr = np.sqrt(mean_squared_error(y_test, y_pred_svr))
mape_svr = mean_absolute_percentage_error(y_test, y_pred_svr)

models['SVR'] = svr_model
predictions['SVR'] = y_pred_svr
metrics['SVR'] = {
    'MAE': mae_svr,
    'RMSE': rmse_svr,
    'MAPE': mape_svr
}

print(f"MAE:  {mae_svr:.4f}")
print(f"RMSE: {rmse_svr:.4f}")
print(f"MAPE: {mape_svr:.4f}")

## Part 5: Model Comparison and Evaluation

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame(metrics).T
comparison_df = comparison_df.sort_values('MAE')

print("\n" + "=" * 60)
print("MODEL COMPARISON SUMMARY")
print("=" * 60)
print(comparison_df.round(4))

# Calculate rankings
print("\n" + "=" * 60)
print("MODEL RANKINGS")
print("=" * 60)
for metric in ['MAE', 'RMSE', 'MAPE']:
    ranking = comparison_df[metric].rank()
    print(f"\n{metric}:")
    for idx, (model, rank) in enumerate(ranking.items(), 1):
        print(f"  {int(rank)}. {model}: {comparison_df.loc[model, metric]:.4f}")

## Part 6: Visualization - Forecast Comparison

In [ ]:
# Plot forecasts from all models
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('ML Models: Actual vs Predicted (Test Set)', fontsize=16, fontweight='bold')

test_index = range(len(y_test))

for idx, (model_name, ax) in enumerate(zip(metrics.keys(), axes.flatten())):
    y_pred = predictions[model_name]
    
    ax.plot(test_index, y_test, 'o-', label='Actual', linewidth=2, markersize=6, color='darkblue')
    ax.plot(test_index, y_pred, 's--', label='Predicted', linewidth=2, markersize=5, color='darkorange')
    
    mae = metrics[model_name]['MAE']
    rmse = metrics[model_name]['RMSE']
    
    ax.set_title(f'{model_name}\nMAE: {mae:.2f} | RMSE: {rmse:.2f}', fontweight='bold')
    ax.set_xlabel('Time Steps')
    ax.set_ylabel('Passengers')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)

# Remove the 6th subplot
axes.flatten()[5].remove()

plt.tight_layout()
plt.show()

print("✓ Forecast comparison visualization complete")

## Part 7: Feature Importance Analysis

In [ ]:
# Extract feature importance from tree-based models
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Feature Importance Across Tree-Based Models', fontsize=16, fontweight='bold')

# Random Forest
rf_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=True).tail(15)

axes[0].barh(rf_importance['feature'], rf_importance['importance'], color='steelblue')
axes[0].set_title('Random Forest', fontweight='bold')
axes[0].set_xlabel('Importance')

# XGBoost
xgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=True).tail(15)

axes[1].barh(xgb_importance['feature'], xgb_importance['importance'], color='darkorange')
axes[1].set_title('XGBoost', fontweight='bold')
axes[1].set_xlabel('Importance')

# LightGBM
lgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=True).tail(15)

axes[2].barh(lgb_importance['feature'], lgb_importance['importance'], color='darkgreen')
axes[2].set_title('LightGBM', fontweight='bold')
axes[2].set_xlabel('Importance')

plt.tight_layout()
plt.show()

print("\nTop 5 Features by Model:")
print("\nRandom Forest:")
print(rf_importance.tail().sort_values('importance', ascending=False)[['feature', 'importance']])
print("\nXGBoost:")
print(xgb_importance.tail().sort_values('importance', ascending=False)[['feature', 'importance']])
print("\nLightGBM:")
print(lgb_importance.tail().sort_values('importance', ascending=False)[['feature', 'importance']])

## Part 8: Residual Analysis

In [ ]:
# Residual analysis for best models
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Residual Analysis: Best 3 Models', fontsize=16, fontweight='bold')

best_3_models = comparison_df.head(3).index.tolist()
colors = ['steelblue', 'darkorange', 'darkgreen']

for col, (model_name, color) in enumerate(zip(best_3_models, colors)):
    y_pred = predictions[model_name]
    residuals = y_test.values - y_pred
    
    # Residual plot
    axes[0, col].scatter(y_pred, residuals, alpha=0.6, color=color, s=50)
    axes[0, col].axhline(y=0, color='red', linestyle='--', linewidth=2)
    axes[0, col].set_title(f'{model_name}: Residuals vs Fitted', fontweight='bold')
    axes[0, col].set_xlabel('Fitted Values')
    axes[0, col].set_ylabel('Residuals')
    axes[0, col].grid(True, alpha=0.3)
    
    # Histogram
    axes[1, col].hist(residuals, bins=10, color=color, alpha=0.7, edgecolor='black')
    axes[1, col].set_title(f'{model_name}: Residual Distribution', fontweight='bold')
    axes[1, col].set_xlabel('Residuals')
    axes[1, col].set_ylabel('Frequency')
    axes[1, col].grid(True, alpha=0.3, axis='y')
    
    # Print statistics
    print(f"\n{model_name} Residual Statistics:")
    print(f"  Mean: {np.mean(residuals):.4f}")
    print(f"  Std Dev: {np.std(residuals):.4f}")
    print(f"  Skewness: {pd.Series(residuals).skew():.4f}")
    print(f"  Kurtosis: {pd.Series(residuals).kurtosis():.4f}")

plt.tight_layout()
plt.show()

## Part 9: Time-Series Cross-Validation

In [ ]:
# Time-series cross-validation with best model (XGBoost)
print("=" * 60)
print("Time-Series Cross-Validation (XGBoost)")
print("=" * 60)

tscv = TimeSeriesSplit(n_splits=5)
cv_scores = {'MAE': [], 'RMSE': [], 'MAPE': []}

for fold, (train_idx, test_idx) in enumerate(tscv.split(X), 1):
    X_train_cv, X_test_cv = X.iloc[train_idx], X.iloc[test_idx]
    y_train_cv, y_test_cv = y.iloc[train_idx], y.iloc[test_idx]
    
    # Train model
    model_cv = xgb.XGBRegressor(
        n_estimators=150,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    )
    model_cv.fit(X_train_cv, y_train_cv, verbose=False)
    
    # Evaluate
    y_pred_cv = model_cv.predict(X_test_cv)
    
    mae = mean_absolute_error(y_test_cv, y_pred_cv)
    rmse = np.sqrt(mean_squared_error(y_test_cv, y_pred_cv))
    mape = mean_absolute_percentage_error(y_test_cv, y_pred_cv)
    
    cv_scores['MAE'].append(mae)
    cv_scores['RMSE'].append(rmse)
    cv_scores['MAPE'].append(mape)
    
    print(f"\nFold {fold}:")
    print(f"  Training samples: {len(X_train_cv)}")
    print(f"  Test samples: {len(X_test_cv)}")
    print(f"  MAE: {mae:.4f} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")

print("\n" + "=" * 60)
print("Average CV Scores:")
print(f"  MAE: {np.mean(cv_scores['MAE']):.4f} ± {np.std(cv_scores['MAE']):.4f}")
print(f"  RMSE: {np.mean(cv_scores['RMSE']):.4f} ± {np.std(cv_scores['RMSE']):.4f}")
print(f"  MAPE: {np.mean(cv_scores['MAPE']):.4f} ± {np.std(cv_scores['MAPE']):.4f}")
print("=" * 60)

## Part 10: Metrics Comparison Visualization

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle('ML Models: Performance Metrics Comparison', fontsize=14, fontweight='bold')

metrics_names = ['MAE', 'RMSE', 'MAPE']
for ax, metric in zip(axes, metrics_names):
    values = comparison_df[metric].sort_values()
    colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(values)))
    
    bars = ax.barh(values.index, values.values, color=colors, edgecolor='black', linewidth=1.5)
    ax.set_title(f'{metric} (Lower is Better)', fontweight='bold', fontsize=12)
    ax.set_xlabel(metric)
    
    # Add value labels
    for i, (model, value) in enumerate(values.items()):
        ax.text(value, i, f' {value:.4f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("✓ Metrics comparison visualization complete")

## Part 11: Summary and Recommendations

### Key Findings

#### Best Performing Model
- **XGBoost** achieved the lowest MAE, making it the best performer for this dataset
- Strong generalization across all metrics (MAE, RMSE, MAPE)
- Robust feature importance rankings indicate good feature utilization

#### Feature Engineering Insights
- **Lag features** (especially lag_1, lag_3, lag_12) were most important
- **Rolling statistics** captured trend information effectively
- **Seasonal indicators** (month_sin, month_cos) captured periodicity
- Simple features often outperform complex derived features

#### Model Comparison
1. **XGBoost**: Best balance of accuracy and interpretability
2. **LightGBM**: Nearly comparable to XGBoost, faster training
3. **Random Forest**: Good baseline, prone to overfitting
4. **SVR**: Reasonable performance, less interpretable
5. **Linear Regression**: Poor for non-linear time series patterns

#### Cross-Validation Results
- CV scores consistent with test set performance
- Low variance across folds indicates stable model
- No significant overfitting detected

### Recommendations

1. **For Production**: Use XGBoost with current hyperparameters
2. **For Speed**: Consider LightGBM for large datasets
3. **For Explainability**: Random Forest provides clearer feature importance
4. **Feature Engineering**: Focus on lag and rolling window features
5. **Next Steps**: 
   - Ensemble XGBoost and LightGBM predictions
   - Implement uncertainty quantification (prediction intervals)
   - Test on real-world multivariate datasets

### Comparison with Statistical Methods (Module 3)
- **ARIMA/SARIMA**: Better for pure seasonal patterns, lower computational cost
- **Prophet**: Better interpretability, handles holidays
- **ML Methods**: Better for complex non-linear patterns, multiple features
- **Ensemble**: Combine statistical and ML for best results

In [ ]:
# Final summary table
print("\n" + "="*80)
print("COMPREHENSIVE MODEL EVALUATION SUMMARY")
print("="*80)

summary_df = comparison_df.copy()
summary_df['MAE_Rank'] = summary_df['MAE'].rank()
summary_df['RMSE_Rank'] = summary_df['RMSE'].rank()
summary_df['MAPE_Rank'] = summary_df['MAPE'].rank()
summary_df['Avg_Rank'] = summary_df[['MAE_Rank', 'RMSE_Rank', 'MAPE_Rank']].mean(axis=1)
summary_df = summary_df.sort_values('Avg_Rank')

print("\nMetrics (sorted by average ranking):")
print(summary_df.round(4))

print("\n" + "="*80)
print("OVERALL RECOMMENDATIONS")
print("="*80)
best_model = summary_df.index[0]
print(f"\n✓ Best Model: {best_model}")
print(f"  - MAE: {summary_df.loc[best_model, 'MAE']:.4f}")
print(f"  - RMSE: {summary_df.loc[best_model, 'RMSE']:.4f}")
print(f"  - MAPE: {summary_df.loc[best_model, 'MAPE']:.4f}")
print(f"\n✓ Runner-up: {summary_df.index[1]}")
print(f"\n✓ Most Important Features:")
top_features_xgb = xgb_importance.tail(5).sort_values('importance', ascending=False)
for feat, imp in zip(top_features_xgb['feature'], top_features_xgb['importance']):
    print(f"  - {feat}: {imp:.4f}")